In [18]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
# Load train and test datasets
train_df = pd.read_csv("/kaggle/input/datascienee/train (15).csv")
test_df = pd.read_csv("/kaggle/input/datascienee/test (15).csv")

# Fill missing content with title if empty
train_df["content"].fillna(train_df["title"], inplace=True)
test_df["content"].fillna(test_df["title"], inplace=True)

<ipython-input-3-e7cdb1bd589e>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["content"].fillna(train_df["title"], inplace=True)
<ipython-input-3-e7cdb1bd589e>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [4]:
# Define stopwords manually
custom_stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
    "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"
])

# Function for text cleaning
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()  # Tokenization
    words = [word for word in words if word not in custom_stopwords]  # Remove stopwords
    return " ".join(words)

# Apply text cleaning
train_df["clean_content"] = train_df["content"].apply(clean_text)
test_df["clean_content"] = test_df["content"].apply(clean_text)

In [5]:
train_df.head(2)

,id,title,content,target,clean_content
0,11098,Getting of the weed,"[Post removed at request of member]\nHi, welco...",suicidal-thoughts-and-self-harm,post removed request member hi welcome immedia...
1,116,Seeking advice on how to face a hospital stay,"Hi NMTB,\nThanks for your post. \nI think a lo...",anxiety,hi nmtb thanks post think lot people terrified...


In [6]:
test_df.head(2)

,id,title,content,clean_content
0,3639,Tailgated,Hello. I was badly tailgated on a busy highway...,hello badly tailgated busy highway today wonde...
1,21493,I am a model and because of mean things my mot...,Morning Scapegoated.\nOk. Scape goated is defi...,morning scapegoated ok scape goated defined pe...


In [7]:

# Split training data
X_train, X_val, y_train, y_val = train_test_split(
    train_df["clean_content"], train_df["target"], test_size=0.2, random_state=42
)

In [8]:
# 1️⃣ **SVM Model (TF-IDF)**
tfidf = TfidfVectorizer(max_features=3000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(test_df["clean_content"])

In [ ]:
# Define the SVM model with different kernels
svm_model = SVC(probability=True)

# Define hyperparameters for tuning
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],  # Trying different kernels
    'C': [0.1, 1, 10],  # Regularization strength
    'gamma': ['scale', 'auto']  # Kernel coefficient
}

# Grid Search for best parameters
grid_search = GridSearchCV(svm_model, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

# Get best model from Grid Search
best_svm_model = grid_search.best_estimator_

# Predictions on validation data
svm_pred = best_svm_model.predict(X_val_tfidf)

# Compute accuracy
svm_acc = accuracy_score(y_val, svm_pred)

print(f"Best SVM Parameters: {grid_search.best_params_}")
print(f"Optimized SVM Accuracy: {svm_acc}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits


In [9]:
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_tfidf, y_train)
svm_pred = svm_model.predict(X_val_tfidf)
svm_acc = accuracy_score(y_val, svm_pred)

In [10]:
svm_acc

0.7271496276235613

In [12]:
# 3️⃣ **GRU Model (Word Embeddings)**
# Tokenization
max_words = 10000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [13]:
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_val_seq = pad_sequences(tokenizer.texts_to_sequences(X_val), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(test_df["clean_content"]), maxlen=max_len)

In [14]:
# GRU Model
gru_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    GRU(64, return_sequences=True),
    GRU(32),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(train_df["target"].unique()), activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:

gru_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train_numeric = y_train.astype('category').cat.codes
y_val_numeric = y_val.astype('category').cat.codes

gru_model.fit(X_train_seq, y_train_numeric, validation_data=(X_val_seq, y_val_numeric), epochs=15, batch_size=64)


Epoch 1/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.3340 - loss: 1.4539 - val_accuracy: 0.5622 - val_loss: 1.1333
Epoch 2/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.6027 - loss: 1.0563 - val_accuracy: 0.6069 - val_loss: 1.0565
Epoch 3/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.6906 - loss: 0.8650 - val_accuracy: 0.6055 - val_loss: 1.0563
Epoch 4/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.7439 - loss: 0.7372 - val_accuracy: 0.5915 - val_loss: 1.0993
Epoch 5/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.7923 - loss: 0.6101 - val_accuracy: 0.5911 - val_loss: 1.1800
Epoch 6/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8329 - loss: 0.4923 - val_accuracy: 0.5915 - val_loss: 1.2647
Epoch 7/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8637 - loss: 0.4108 - val_accuracy: 0.5845 - val_loss: 1.3819
Epoch 8/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8943 - loss: 0.3295 - val_acc

In [16]:

# Predict on test set
svm_test_pred = svm_model.predict(X_test_tfidf)
gru_test_pred = gru_model.predict(X_test_seq)

# Convert GRU predictions to labels
gru_test_pred_labels = np.argmax(gru_test_pred, axis=1)
target_classes = y_train.astype('category').cat.categories
gru_test_pred_final = [target_classes[i] for i in gru_test_pred_labels]

77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [17]:

# Create Submission Files
def save_submission(predictions, model_name):
    submission = test_df[["id"]].copy()
    submission["Target"] = predictions
    submission.to_csv(f"submission_{model_name}.csv", index=False)
    print(f"Submission file generated: submission_{model_name}.csv")

save_submission(svm_test_pred, "SVM")
save_submission(gru_test_pred_final, "GRU")


Submission file generated: submission_SVM.csv
Submission file generated: submission_GRU.csv
